# Extracción de Datos

Nuestros datos son provienen de varias pruebas con diferentes ciclos de secado de ropa, nuestro objetivo sera identificar el ciclo de secado que se esta ejecutando.

In [160]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Originalmente la Data se encontraba en un archivo de Excel, esto la hacia dificil de manipular y de entender, por lo que anterior mente, la transforme en varios archivos de CSV con los siguientes **Features**:
* *Time(min)* $\to$ nos indica el tiempo en fracciones de minuto
* *RMC* $\to$ es el porcentaje de humedad dentro de la secadora, 100-45% equivale a que la ropa esta mojada, 0-2% la ropa esta seca.
* *RDB* $\to$ no recuerdo, deberia preguntar.
* *Inlet* $\to$ Es la temperatura (F°) del aire dentro de la secadora.
* *Outlet* $\to$ Es la temperatura (F°) del aire que esta saliendo de la secadora

Entre muchos otros parametros.

In [161]:
# Tomamos nuestro Data Set
df_mixed = pd.read_csv('Data_Sets\Jeans.csv')
df_cottons = pd.read_csv('Data_Sets\Mixed.csv')

n = 150 # los primeros 3 minutos no se consideran
df_mixed = df_mixed[df_mixed['Time(min)']>n]
df_cottons = df_cottons[df_cottons['Time(min)']>n]

# El peso es un factor importante en el comportamiento
# de la secadora.
w1 = 16 # Peso en lbs
# incluir peso 1 y peso 2
df_mixed = df_mixed[(df_mixed['Weight']==w1)]# | df_mixed['Weight']==w2)]
df_cottons = df_cottons[(df_cottons['Weight']==w1)]# | (df_cottons['Weight']==w2)] 

# Comviene quitar los valores de delta time en 0
# es cuando la secadora esta apagada
# df_mixed = df_mixed[df_mixed['Delta time']!=0]
# df_cottons = df_cottons[df_cottons['Delta time']!=0]


df_mixed['Class']= 0 # 0 para mixed
df_cottons['Class']= 1 # 1 para cottons


# Concatenar los dataframes para poder hacer la clasificacion
df = pd.concat([df_mixed, df_cottons], axis=0)
# Shuffle 
df = df.sample(frac=1).reset_index(drop=True)
# Revisar la informacion del dataframe
df.head()

<>:2: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\J'
<>:3: SyntaxWarning: invalid escape sequence '\M'
C:\Users\PC\AppData\Local\Temp\ipykernel_18660\420197573.py:2: SyntaxWarning: invalid escape sequence '\J'
  df_mixed = pd.read_csv('Data_Sets\Jeans.csv')
C:\Users\PC\AppData\Local\Temp\ipykernel_18660\420197573.py:3: SyntaxWarning: invalid escape sequence '\M'
  df_cottons = pd.read_csv('Data_Sets\Mixed.csv')


,Time(min),RMC,RDB,Energy,Potenza,Filtered,Smooth,Outlet,Inlet,T-A amb,RH amb,Weight,Restriction,Class,Segment
0,330.126,18.64,91,5530.8,263.5,2970,2954,96,171,76.4,49.2,16,1.5,0,6
1,245.767,24.12,90,4884.4,264.1,986,1094,95,167,74.2,55.0,16,1.5,1,6
2,215.957,13.58,109,5929.0,245.8,5217,5406,110,214,77.4,44.4,16,1.5,0,2
3,292.876,27.34,91,4947.1,251.3,927,1055,96,168,75.2,51.1,16,1.5,0,6
4,327.460,19.24,90,5473.4,254.8,2903,2995,96,168,75.6,52.8,16,1.5,0,6


In [162]:
# Vamos a dropear las columnas que no nos interesan

# La Restriccion aunque parace afectar
# realmente tiene mucha STD 
# por lo que no se considera un factor importante
# por lo que para poder ocupar el maypr numero de datos
# lo dejamos fuera



# La energia no es un factor importante al igual que la Potenza
# por lo que se eliminan
df = df.drop(columns=['Segment','Restriction','Energy','Potenza','Filtered','Smooth','T-A amb','RH amb'])
df.head()


,Time(min),RMC,RDB,Outlet,Inlet,Weight,Class
0,330.126,18.64,91,96,171,16,0
1,245.767,24.12,90,95,167,16,1
2,215.957,13.58,109,110,214,16,0
3,292.876,27.34,91,96,168,16,0
4,327.460,19.24,90,96,168,16,0


In [163]:
# Me gustaria ver que tan cargado esta nuestro data set
# para ver si es necesario hacer un balanceo
print(df['Class'].value_counts())


Class
0    47818
1    39971
Name: count, dtype: int64


In [164]:
# Vamos a hacer un balanceo de los datos
# para que no haya un sesgo en el modelo
min_class = df['Class'].value_counts().min()
df = df.groupby('Class').apply(lambda x: x.sample(min_class)).reset_index(drop=True)
print(df['Class'].value_counts())

Class
0    39971
1    39971
Name: count, dtype: int64


C:\Users\PC\AppData\Local\Temp\ipykernel_18660\3710118729.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Class').apply(lambda x: x.sample(min_class)).reset_index(drop=True)


In [165]:

# Agregar relacion entre las variables
df['out/in'] = df['Outlet']/df['Inlet']
# Agregar el diferencial de Inlet
df['Delta T in'] = (df['Inlet'] - df['Outlet'].shift(1))**2
# Agregar el diferencial de Outlet
df['Delta T out'] = (df['Outlet'] - df['Outlet'].shift(1))**2

df['Delta RDBOutlet'] = (df['Outlet']-df['RDB'])**2
# Agregar el cuadrado
df['Inlet']= df['Inlet']**2
df['Outlet']= df['Outlet']**2


In [166]:
# Ahora vamos a guardar nuestro data set en un CSV
df.to_csv('Data_Sets\Processed.csv', index=False)

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\PC\AppData\Local\Temp\ipykernel_18660\3057385823.py:2: SyntaxWarning: invalid escape sequence '\P'
  df.to_csv('Data_Sets\Processed.csv', index=False)
